In [1]:
import os
import random
import pickle
from joblib import Parallel, delayed
import glob
import torch
import networkx as nx

# === Standard Library ===
import math
import random
import time
import heapq
import itertools
from collections import defaultdict, deque
from itertools import combinations
from typing import Any, Tuple, Dict, List, Set, Sequence, Union

# === Third-Party Libraries ===

# --- Scientific Computing ---
import numpy as np
import pandas as pd
import scipy.sparse as sp
from scipy.sparse    import coo_matrix
from scipy.optimize import linprog

# --- Plotting ---
import matplotlib.pyplot as plt

# --- Parallel Processing ---
from joblib import Parallel, delayed
from tqdm import tqdm

# --- Graph Processing ---
import networkx as nx

# --- JIT Compilation ---
from numba import njit, prange

from torch.utils.data import Dataset, DataLoader
from torch_geometric.loader import NeighborLoader
from torch_geometric.utils import from_networkx
from torch_geometric.nn import SAGEConv


# --- Model definition ---
import torch.nn as nn
import torch.nn.functional as F


/home/tuguldurb/Development/Research/SCNDP/lib/python3.10/site-packages/torch/__config__.py:9: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._show_config()


In [2]:
import os
import torch

base_folder = "/home/tuguldurb/Development/Research/SCNDP/src/SCNDP/src/extension/learning/notebooks/gnn/data/graphs_labels"         # Folder with .pt files
output_folder = "/home/tuguldurb/Development/Research/SCNDP/src/SCNDP/src/extension/learning/notebooks/gnn/data/graphs_labels_txt"   # Destination folder for .txt files

for root, _, files in os.walk(base_folder):
    for file in files:
        if file.endswith(".pt"):
            pt_path = os.path.join(root, file)
            
            # Load tensor or model (depending on format)
            try:
                content = torch.load(pt_path, map_location='cpu')
            except Exception as e:
                print(f"Skipping {pt_path} due to load error: {e}")
                continue
            
            # Generate corresponding txt path
            rel_path = os.path.relpath(pt_path, base_folder)
            txt_path = os.path.join(output_folder, os.path.splitext(rel_path)[0] + ".txt")
            os.makedirs(os.path.dirname(txt_path), exist_ok=True)

            # Write to text file
            try:
                with open(txt_path, "w") as f:
                    f.write(str(content))
            except Exception as e:
                print(f"Error writing {txt_path}: {e}")

In [ ]:
import os
import shutil
from pathlib import Path

# Adjust these paths:
input_dir = Path("/home/tuguldurb/Development/Research/SCNDP/src/SCNDP/src/extension/learning/notebooks/gnn/data/labels/test_large")

output_base = Path("/home/tuguldurb/Development/Research/SCNDP/src/SCNDP/src/extension/learning/notebooks/gnn/data/labels/test_large_separate")

# Make sure output base exists
output_base.mkdir(parents=True, exist_ok=True)

for pkl_file in input_dir.glob("*.pt"):
    # Extract the prefix before the first underscore or other separator
    # e.g. "ER_graph1.pkl" → "ER"
    prefix = pkl_file.stem.split("_", 1)[0]
    
    # Define the target folder for this prefix
    target_folder = output_base / prefix
    target_folder.mkdir(exist_ok=True)
    
    # Move the file
    shutil.move(str(pkl_file), str(target_folder / pkl_file.name))

    print(f"Moved {pkl_file.name} → {target_folder}/")


Moved SW_sz500_sp5_rp0.7_test_large_1.pkl → /home/tuguldurb/Development/Research/SCNDP/src/SCNDP/src/extension/learning/notebooks/gnn/data/graphs/test_large_separate/SW/
Moved SW_sz500_sp5_rp0.1_test_large_0.pkl → /home/tuguldurb/Development/Research/SCNDP/src/SCNDP/src/extension/learning/notebooks/gnn/data/graphs/test_large_separate/SW/
Moved ER_sz500_sp0.0443_rp0.3_test_large_1.pkl → /home/tuguldurb/Development/Research/SCNDP/src/SCNDP/src/extension/learning/notebooks/gnn/data/graphs/test_large_separate/ER/
Moved SW_sz200_sp4_rp0.4_test_large_0.pkl → /home/tuguldurb/Development/Research/SCNDP/src/SCNDP/src/extension/learning/notebooks/gnn/data/graphs/test_large_separate/SW/
Moved SW_sz300_sp5_rp0.6_test_large_0.pkl → /home/tuguldurb/Development/Research/SCNDP/src/SCNDP/src/extension/learning/notebooks/gnn/data/graphs/test_large_separate/SW/
Moved ER_sz200_sp0.0667_rp0.2_test_large_1.pkl → /home/tuguldurb/Development/Research/SCNDP/src/SCNDP/src/extension/learning/notebooks/gnn/data/g